In [1]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder \
                    .master('local[*]') \
                    .appName('aggregations_demo') \
                    .getOrCreate()

In [6]:
raw_df = spark.read \
                .format('csv') \
                .option('inferSchema', True) \
                .option('header', True) \
                .load(r'C:\Users\chasurag\Documents\my repos\data\invoices.csv')

raw_df.show(10)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|     NULL|WHITE HANGING HEA...|       6|01-12-2010 8.26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|01-12-2010 8.26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|01-12-2010 8.26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|01-12-2010 8.

In [10]:
raw_df.select(count(expr('*')).alias('total_rows'), \
                sum(col('quantity')).alias('total_quantity'), \
                round(avg('UnitPrice'), 2).alias('avg_price'), \
                count_distinct('invoiceno').alias('total_distinct_invoices')).show()

+----------+--------------+---------+-----------------------+
|total_rows|total_quantity|avg_price|total_distinct_invoices|
+----------+--------------+---------+-----------------------+
|    541909|       5176450|     4.61|                  25900|
+----------+--------------+---------+-----------------------+



In [16]:
raw_df.groupBy('country', 'invoiceno') \
        .agg(
                sum('quantity').alias('total_quantity'), \
                round(sum(col('quantity') * col('unitprice')), 2).alias('invoice_value')  
        ) \
        .show()

+--------------+---------+--------------+-------------+
|       country|invoiceno|total_quantity|invoice_value|
+--------------+---------+--------------+-------------+
|United Kingdom|   536446|           329|       440.89|
|United Kingdom|   536508|           216|       155.52|
|United Kingdom|   537018|            -3|          0.0|
|United Kingdom|   537401|           -24|          0.0|
|United Kingdom|   537811|            74|       268.86|
|United Kingdom|  C537824|            -2|        -14.9|
|United Kingdom|   538895|           370|       247.38|
|United Kingdom|   540453|           341|       302.45|
|United Kingdom|   536627|            64|        306.2|
|United Kingdom|   537224|           700|      1415.97|
|United Kingdom|   537230|             1|         2.95|
|United Kingdom|   537682|            72|         99.6|
|United Kingdom|  C538749|            -1|        -9.95|
|        Israel|  C539037|           -56|      -227.44|
|United Kingdom|   539076|            96|       